In [47]:
%pip install -U langchain-community
%pip install -qU langchain-openai
%reset -f

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [37]:
from typing import Any

from langchain.agents import initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.tools import BaseTool

# Define the tools with type annotations for name and description
class CodeLinterTool(BaseTool):
    name: str = "CodeLinter" 
    description: str = "Analyzes the provided C# code for linting issues, best practices, and suggests improvements."

    def _run(self, code: str) -> str:
        # Example placeholder logic for code linting
        return f"Linting results: No issues found for the given C# code."

    def _arun(self, *args: Any, **kwargs: Any) -> str:
        raise NotImplementedError("This tool does not support async execution.")


class CodeRefactorerTool(BaseTool):
    name: str = "CodeRefactorer" 
    description: str = "Refactors the provided C# code to be more efficient and follow best practices."

    def _run(self, code: str) -> str:
        # Example placeholder logic for refactoring
        refactored_code = code.replace('string', 'var').replace('int[]', 'var')  # Just an example
        return f"Refactored code:\n{refactored_code}"

    def _arun(self, *args: Any, **kwargs: Any) -> str:
        raise NotImplementedError("This tool does not support async execution.")


class DebuggerTool(BaseTool):
    name: str = "Debugger" 
    description: str = "Identifies logical errors in the provided C# code and suggests possible fixes."

    def _run(self, code: str) -> str:
        # Example placeholder logic for debugging
        return f"Debugging results: Found a possible NullReferenceException on line 15."

    def _arun(self, *args: Any, **kwargs: Any) -> str:
        raise NotImplementedError("This tool does not support async execution.")

In [38]:
from dotenv import load_dotenv
import os

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

In [39]:

# Define the list of tools with proper type annotations
tools = [
    CodeLinterTool(),
    CodeRefactorerTool(),
    DebuggerTool()
]

# OPENAI_API_KEY env variable must be defined
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
llm = ChatOpenAI(model="gpt-4", temperature=0.5)  # Use GPT-4 model
agent = initialize_agent(
    tools=tools,
    llm=llm,
    verbose=True,
    memory=memory
)

In [41]:
# User input to the ReAct agent
user_input = """
I have the following C# code:
using System;

class Program { 
    static void Main() { 
        string name = "John Doe"; Console.WriteLine("Hello, " + name);
        int[] numbers = {1, 2, 3, 4, 5};
        foreach (var number in numbers)
        {
            Console.WriteLine(number);
        }
    }
}

Please check for any issues, refactor the code for best practices, and debug it.
"""


In [42]:
# Run the agent
response = agent.invoke(user_input)
print(response)



> Entering new AgentExecutor chain...
I need to check if there are any linting issues, refactor the code for best practices, and debug it if necessary.
Action: CodeLinter
Action Input: The provided C# code
Observation: Linting results: No issues found for the given C# code.
Thought:The code does not have any linting issues. Now I need to refactor the code for best practices.
Action: CodeRefactorer
Action Input: The provided C# code
Observation: Refactored code:
The provided C# code
Thought:The code has been refactored. Now I need to debug the code to check for any logical errors.
Action: Debugger
Action Input: The provided C# code
Observation: Debugging results: Found a possible NullReferenceException on line 15.
Thought:There is a potential NullReferenceException on line 15. I need to check this line and fix the issue.
Action: CodeRefactorer
Action Input: The provided C# code with focus on line 15
Observation: Refactored code:
The provided C# code with focus on line 15
Thought:I hav